In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import os

from keras.layers import Concatenate, Input, Dense, Embedding, Flatten, Dropout, BatchNormalization, SpatialDropout1D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.models import Model
from keras.optimizers import  Adam
import keras.backend as k

# import matplotlib.pyplot as plt
# %matplotlib inline
# from tqdm import tqdm_notebook
from sklearn.preprocessing import StandardScaler
# from sklearn.svm import NuSVR, SVR
from sklearn.metrics import mean_absolute_error
# pd.options.display.precision = 15
from category_encoders.cat_boost import CatBoostEncoder

# import lightgbm as lgb
# import xgboost as xgb
# import time
# import datetime
# from catboost import CatBoostRegressor
# from sklearn.preprocessing import LabelEncoder
# from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold, GroupKFold, GridSearchCV, train_test_split, TimeSeriesSplit
# from sklearn import metrics
# from sklearn import linear_model
import gc
import pickle
# import seaborn as sns
# import warnings
# warnings.filterwarnings("ignore")

# import eli5
# import shap
# from IPython.display import HTML
# import json
# import altair as alt

# import networkx as nx
# import matplotlib.pyplot as plt

gc.collect()
# alt.renderers.enable('notebook')

Using TensorFlow backend.
c:\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as

0

In [2]:
main_path = r'd:\Documents\Private\Kaggle\IEEEFraud'

In [3]:
import sys
sys.path.append(main_path)
from BayDS import *

In [4]:
experiment_name = '31.08'
main_learning_folder = main_path+'/Snapshots/'+experiment_name


In [5]:
#start here
y = pd.read_pickle(f'{main_learning_folder}/y.pkl')
X = pd.read_pickle(f'{main_learning_folder}/X_encoded_scaled.pkl').astype(np.float32)
test = pd.read_pickle(f'{main_learning_folder}/test_encoded_scaled.pkl').astype(np.float32)
gc.collect()

51

In [6]:
# Setting model_folder
model_name = 'keras-3'
model_folder = f'{main_learning_folder}/{model_name}'
if not os.path.exists(model_folder):
    os.makedirs(model_folder)

In [7]:
n_fold = 5
# folds = TimeSeriesSplit(n_splits=n_fold)
folds = KFold(n_splits=n_fold)
# folds = GroupKFold(n_splits=5)
# groups = pd.read_pickle('./groups.pkl').values

In [8]:
y.value_counts()

0    569877
1     20663
Name: isFraud, dtype: int64

In [9]:
def NNModel_maker():
    k.clear_session()
    
#     categorical_inputs = []
#     for cat in categorical:
#         categorical_inputs.append(Input(shape=[1], name=cat))

#     categorical_embeddings = []
#     for i, cat in enumerate(categorical):
#         categorical_embeddings.append(
#             Embedding(category_counts[cat], int(np.log1p(category_counts[cat]) + 1), name = cat + \
#                       "_embed")(categorical_inputs[i]))

#     categorical_logits = Concatenate(name = "categorical_conc")([Flatten()(SpatialDropout1D(.1)(cat_emb)) for cat_emb in categorical_embeddings])
# 
    numerical_inputs = Input(shape=[X.shape[1]], name = 'all')
    numerical_logits = Dropout(.3)(numerical_inputs)
  
    x = numerical_logits

    x = Dense(200, activation = 'relu')(x)
    x = Dropout(.3)(x)
    x = Dense(100, activation = 'relu')(x)
    x = Dropout(.3)(x)
    x = BatchNormalization()(x)    
    
    out = Dense(1, activation = 'sigmoid')(x)    

    model = Model(inputs= [numerical_inputs],outputs=out)
    loss = "binary_crossentropy"
    model.compile(optimizer=Adam(lr = 0.0003), loss = loss)
    return model


params = {
    'batch_size': 8000,
    'epochs': 60,
    'verbose': True,
         }
train_options = {
    "model_type":'keras',
    "params": params,
    "eval_metric":'auc',
    'averaging': 'usual',
    'use_groups': False,
    'fold_name': folds.__class__.__name__,
    'n_splits': n_fold
   
}

In [10]:
with open(f'{model_folder}/training_params.json', 'w') as f:
    q = json.dumps(train_options,indent=2)
    f.write(q)


In [11]:
NNModel_maker().save(f'{model_folder}/keras.mdl')





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [12]:
gc.collect()
result_dict_keras = train_model_classification(model=NNModel_maker, 
                                             X=X,
                                             X_test=test,
                                             y=y, params=params, folds=folds,
                                             model_type=train_options['model_type'], 
                                             eval_metric=train_options['eval_metric'],
                                             plot_feature_importance=True,
                                             averaging=train_options['averaging'],
                                             groups=None)

Fold 1 started at Tue Sep  3 19:18:11 2019
Train on 472432 samples, validate on 118108 samples
Epoch 1/60
472432/472432 [==============================] - 16s 35us/step - loss: 0.7004 - val_loss: 0.7936
Epoch 2/60
472432/472432 [==============================] - 15s 32us/step - loss: 0.5431 - val_loss: 0.6032
Epoch 3/60
472432/472432 [==============================] - 16s 33us/step - loss: 0.4225 - val_loss: 0.4820
Epoch 4/60
472432/472432 [==============================] - 15s 32us/step - loss: 0.3242 - val_loss: 0.3987
Epoch 5/60
472432/472432 [==============================] - 15s 32us/step - loss: 0.2539 - val_loss: 0.3143
Epoch 6/60
472432/472432 [==============================] - 15s 32us/step - loss: 0.2074 - val_loss: 0.2579
Epoch 7/60
472432/472432 [==============================] - 17s 35us/step - loss: 0.1754 - val_loss: 0.2498
Epoch 8/60
472432/472432 [==============================] - 17s 37us/step - loss: 0.1550 - val_loss: 0.1974
Epoch 9/60
472432/472432 [===============

Epoch 14/60
472432/472432 [==============================] - 17s 35us/step - loss: 0.1096 - val_loss: 0.1113
Epoch 15/60
472432/472432 [==============================] - 16s 35us/step - loss: 0.1071 - val_loss: 0.1087
Epoch 16/60
472432/472432 [==============================] - 15s 33us/step - loss: 0.1042 - val_loss: 0.1067
Epoch 17/60
472432/472432 [==============================] - 15s 33us/step - loss: 0.1021 - val_loss: 0.1050
Epoch 18/60
472432/472432 [==============================] - 16s 33us/step - loss: 0.0993 - val_loss: 0.1031
Epoch 19/60
472432/472432 [==============================] - 15s 33us/step - loss: 0.0968 - val_loss: 0.1019
Epoch 20/60
472432/472432 [==============================] - 16s 33us/step - loss: 0.0950 - val_loss: 0.1013
Epoch 21/60
472432/472432 [==============================] - 15s 33us/step - loss: 0.0926 - val_loss: 0.0976
Epoch 22/60
472432/472432 [==============================] - 15s 33us/step - loss: 0.0920 - val_loss: 0.0963
Epoch 23/60
472432/

472432/472432 [==============================] - 15s 33us/step - loss: 0.0890 - val_loss: 0.0908
Epoch 28/60
472432/472432 [==============================] - 16s 33us/step - loss: 0.0886 - val_loss: 0.0900
Epoch 29/60
472432/472432 [==============================] - 15s 32us/step - loss: 0.0874 - val_loss: 0.0887
Epoch 30/60
472432/472432 [==============================] - 15s 33us/step - loss: 0.0868 - val_loss: 0.0878
Epoch 31/60
472432/472432 [==============================] - 15s 32us/step - loss: 0.0865 - val_loss: 0.0876
Epoch 32/60
472432/472432 [==============================] - 16s 34us/step - loss: 0.0857 - val_loss: 0.0865
Epoch 33/60
472432/472432 [==============================] - 19s 41us/step - loss: 0.0851 - val_loss: 0.0863
Epoch 34/60
472432/472432 [==============================] - 17s 37us/step - loss: 0.0845 - val_loss: 0.0867
Epoch 35/60
472432/472432 [==============================] - 16s 35us/step - loss: 0.0848 - val_loss: 0.0859
Epoch 36/60
472432/472432 [====

472432/472432 [==============================] - 16s 35us/step - loss: 0.0806 - val_loss: 0.0855
Epoch 41/60
472432/472432 [==============================] - 16s 35us/step - loss: 0.0800 - val_loss: 0.0852
Epoch 42/60
472432/472432 [==============================] - 16s 35us/step - loss: 0.0800 - val_loss: 0.0846
Epoch 43/60
472432/472432 [==============================] - 16s 35us/step - loss: 0.0794 - val_loss: 0.0842
Epoch 44/60
472432/472432 [==============================] - 17s 35us/step - loss: 0.0789 - val_loss: 0.0840
Epoch 45/60
472432/472432 [==============================] - 17s 35us/step - loss: 0.0786 - val_loss: 0.0842
Epoch 46/60
472432/472432 [==============================] - 18s 39us/step - loss: 0.0782 - val_loss: 0.0838
Epoch 47/60
472432/472432 [==============================] - 17s 35us/step - loss: 0.0783 - val_loss: 0.0832
Epoch 48/60
472432/472432 [==============================] - 17s 35us/step - loss: 0.0778 - val_loss: 0.0836
Epoch 49/60
472432/472432 [====

472432/472432 [==============================] - 16s 33us/step - loss: 0.0775 - val_loss: 0.0809
Epoch 54/60
472432/472432 [==============================] - 15s 33us/step - loss: 0.0770 - val_loss: 0.0812
Epoch 55/60
472432/472432 [==============================] - 15s 33us/step - loss: 0.0772 - val_loss: 0.0812
Epoch 56/60
472432/472432 [==============================] - 15s 33us/step - loss: 0.0765 - val_loss: 0.0808
Epoch 57/60
472432/472432 [==============================] - 15s 32us/step - loss: 0.0761 - val_loss: 0.0809
Epoch 58/60
472432/472432 [==============================] - 15s 32us/step - loss: 0.0758 - val_loss: 0.0801
Epoch 59/60
472432/472432 [==============================] - 15s 32us/step - loss: 0.0758 - val_loss: 0.0803
Epoch 60/60
118108/118108 [==============================] - 1s 6us/step
Fold 4. auc: 0.9297.
CV mean score: 0.9336, std: 0.0074.


In [13]:
sub = pd.read_csv(f'../../data/sample_submission.csv')

In [14]:
sub['isFraud'] = result_dict_keras['prediction']
sub.to_csv('ieee_nn.csv', index=False)

In [15]:
result_dict_keras['prediction'].shape

(506691, 1)

In [16]:
test.shape

(506691, 789)

In [17]:
import pickle
with open(f'{model_folder}/results_dict.pkl', 'wb') as f:
#     q = json.dumps(result_dict_lgb,indent=2)
    pickle.dump(result_dict_keras,f)
#     f.write(q)